# Visualizing income inequality using PPP-adjusted GDP data

**2022-01-29**

- Reference: https://core-econ.org/the-economy/book/text/01.html#11-income-inequality
- Data: https://jackblun.github.io/Globalinc/

In [19]:
import pandas as pd
import numpy as np
from bokeh.io import output_notebook, show

output_notebook()
pd.options.display.min_rows = 50

Loading BokehJS ...

## Loading the data

In [26]:
data = pd.read_csv("data/GCIPrawdata.csv", header=2)
income = data.melt(
    id_vars=['Country', 'Year'],
    value_vars=[f'Decile {i} Income' for i in range(1, 11)],
    value_name='Income',
    var_name='Decile',
)
income.Decile = income.Decile.str.split(expand=True)[1].astype(int)
income

,Country,Year,Decile,Income
0,Afghanistan,1980,1,206
1,Afghanistan,1981,1,212
2,Afghanistan,1982,1,221
3,Afghanistan,1983,1,238
4,Afghanistan,1984,1,249
5,Afghanistan,1985,1,256
6,Afghanistan,1986,1,268
7,Afghanistan,1987,1,243
8,Afghanistan,1988,1,223
9,Afghanistan,1989,1,202


## Inequality by max and min GDP deciles

In [29]:
def get_inequality(data):
    data = data.set_index(['Country', 'Year', 'Decile'])
    rich = data.xs(10, level='Decile').Income
    poor = data.xs( 1, level='Decile').Income
    return (rich / poor).to_frame('Inequality').reset_index()

inequality = get_inequality(income)
inequality

,Country,Year,Inequality
0,Afghanistan,1980,17.194175
1,Afghanistan,1981,17.240566
2,Afghanistan,1982,17.257919
3,Afghanistan,1983,17.247899
4,Afghanistan,1984,17.232932
5,Afghanistan,1985,17.214844
6,Afghanistan,1986,17.261194
7,Afghanistan,1987,17.267490
8,Afghanistan,1988,17.233184
9,Afghanistan,1989,17.232673


In [28]:
from bokeh.models import ColumnDataSource
from bokeh.plotting import figure


def plot_inequality(data):
    data = (
        data[['Country', 'Inequality']]
        .copy()
        .dropna()
        .sort_values('Inequality')
    )
    data.Inequality = data.Inequality.round(1).apply('{:.1f}'.format)
    data.Country = data.Country.astype(str)
    plot = figure(
        sizing_mode='stretch_both',
        height=2500,
        toolbar_location=None,
        y_range=data.Country,
        tooltips=[("Inequality", "@Inequality"), ("Country", "@Country")],
    )
    plot.hbar(
        y="Country",
        right="Inequality",
        source=ColumnDataSource(data),
        height=2/3,
    )
    plot.ygrid.grid_line_color = None
    plot.outline_line_color = None
    show(plot)

In [30]:
from IPython.display import display, Markdown


for year in (1980, 1990, 2014):
    display(Markdown(f"### {year}"))
    plot_inequality(inequality[inequality.Year == year])

### 1980

### 1990

### 2014